In [ ]:
import pandas as pd
import joblib
import os
from sklearn.preprocessing import StandardScaler

# Step 1: Load the Latest Model Metadata
registry_file = "../data/model/model_registry.csv"
if not os.path.exists(registry_file):
    raise FileNotFoundError(f"Model registry not found at {registry_file}")

registry_df = pd.read_csv(registry_file)

# Get the latest model entry
latest_entry = registry_df.iloc[-1]
model_name = latest_entry["model_name"]
model_path = latest_entry["model_path"]

if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

print(f"Loading the latest model: {model_name}")
svm_model = joblib.load(model_path)

# Step 2: Load New Data for Prediction
new_data = pd.read_csv("../data/raw/new_data.csv")
print("New data loaded.")

# Preprocess New Data
numeric_cols = new_data.select_dtypes(include=['number']).columns
non_numeric_cols = new_data.select_dtypes(exclude=['number']).columns

new_data[numeric_cols] = new_data[numeric_cols].fillna(new_data[numeric_cols].mean())
for col in non_numeric_cols:
    new_data[col] = new_data[col].fillna(new_data[col].mode()[0])

new_data = pd.get_dummies(new_data, drop_first=True)
missing_cols = set(X.columns) - set(new_data.columns)
for col in missing_cols:
    new_data[col] = 0
new_data = new_data[X.columns]

scaler = StandardScaler()
new_data_scaled = scaler.fit_transform(new_data)

# Step 3: Make Predictions
predictions = svm_model.predict(new_data_scaled)
print("Predictions completed.")
print(predictions)
